In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
%run ../../../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
create_temp_view_with_clean_columns(
    spark,
    catalog_name,
    'udp_wcm_bronze_sap_bw',
    'sap_bw_wcm_avccmdm432',
    proc_date,
    'temp_sap_avccmdm432'
)

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_silver_sap_bw.sap_avccmdm432 (
  lifnr STRING,
  ltsnr STRING,
  ernam STRING,
  erdat DATE,
  spras STRING,
  ltsbz STRING,
  minbw DOUBLE,
  waers STRING,
  proc_date TIMESTAMP
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
""")

In [0]:
spark.sql(f"""
INSERT INTO {catalog_name}.udp_wcm_silver_sap_bw.sap_avccmdm432
SELECT 
  LIFNR, 
  LTSNR, 
  ERNAM, 
  TO_DATE(ERDAT, 'yyyyMMdd') AS ERDAT, 
  SPRAS, 
  LTSBZ, 
  MINBW, 
  WAERS, 
  PROC_DATE 
FROM temp_sap_avccmdm432
""")
